In [7]:
# Imports
import os
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
from sec_api import QueryApi
queryApi = QueryApi(api_key="43d18e4ffedde4ca940e46ce210780daa0b7de17674abb369b5a8ef9fb3b69a1")

In [9]:
ticker_list = pd.read_csv("./Data/Cleaned_Data/Ticker_library.csv")["Ticker"].to_list()

In [10]:
def SEC_query(ticker_list,filingType):
    file_dict = {}
    for ticker in ticker_list:
        query = {
            "query": { "query_string": { 
                "query": f"ticker:{ticker} AND filedAt:"+"{2012-06-01 TO 2022-06-01} AND formType:\""+f"{filingType}\"" 
            } },
            "from": "0",
            "size": "60",
            "sort": [{ "filedAt": { "order": "desc" } }]
        }
        filings = queryApi.get_filings(query)
        file_dict[ticker] = pd.json_normalize(filings, record_path =['filings'])
    return file_dict

In [11]:
file_10q_df = SEC_query(ticker_list,'10-Q')
file_10k_df = SEC_query(ticker_list,'10-K')

In [16]:
url_data = {
    "ticker":[],
    "periodOfReport":[],
    "linkToFilingDetails":[],
    "formType":[],
    "cik":[]
    }
def url_clean(file_10q_df):
    for ticker in ticker_list:
        url_data["ticker"].extend(file_10q_df[ticker]["ticker"].to_list())
        url_data["periodOfReport"].extend(file_10q_df[ticker]["periodOfReport"].to_list())
        url_data["linkToFilingDetails"].extend(file_10q_df[ticker]["linkToFilingDetails"].to_list())
        url_data["formType"].extend(file_10q_df[ticker]["formType"].to_list())
        url_data["cik"].extend(file_10q_df[ticker]["cik"].to_list())
url_clean(file_10q_df)
url_clean(file_10k_df)

url_df = pd.DataFrame(url_data)

In [12]:
from sec_api import ExtractorApi
sec_api = os.getenv("sec_api_key")
extractorApi = ExtractorApi("43d18e4ffedde4ca940e46ce210780daa0b7de17674abb369b5a8ef9fb3b69a1")

In [34]:
def sec_text_pull(url,filingtype):
    url = str(url)
    data = {}
    if (filingtype == "10-Q"):
        data['Business']=pd.NA
        data['Risk Factors']=extractorApi.get_section(url, "part2item1A", "text")
        data['Management’s Discussion and Analysis of Financial Condition and Results of Operations']=extractorApi.get_section(url, "part1item2", "text")
    else:
        data['Business']=extractorApi.get_section(url, "1", "text")
        data['Risk Factors']=extractorApi.get_section(url, "1A", "text")
        data['Management’s Discussion and Analysis of Financial Condition and Results of Operations']=extractorApi.get_section(url, "7", "text")
    return data

In [64]:
def pull_sec_from_df(df:pd.DataFrame):
    data = {
        "Business":[],
        "Risk Factors":[],
        'Management’s Discussion and Analysis of Financial Condition and Results of Operations':[]
    }

    for rowNumber in range(len(df.index)):
        pull = sec_text_pull(df['linkToFilingDetails'].loc[rowNumber],df['formType'].loc[rowNumber])
        data['Business'].append(pull['Business'])
        data['Risk Factors'].append(pull['Risk Factors'])
        data['Management’s Discussion and Analysis of Financial Condition and Results of Operations'].append(
            pull['Management’s Discussion and Analysis of Financial Condition and Results of Operations'])
    return pd.DataFrame(data)

In [65]:
sec_text = pull_sec_from_df(url_df)

In [66]:
sec_text.to_csv('.\Data\Cleaned_Data\sec_text_pulls.csv')

In [67]:
sec_text_df = pd.concat([url_df,sec_text],axis=1)
sec_text_df

,ticker,periodOfReport,linkToFilingDetails,formType,cik,Business,Risk Factors,Management’s Discussion and Analysis of Financial Condition and Results of Operations
0,NFLX,2022-03-31,https://www.sec.gov/Archives/edgar/data/106528...,10-Q,1065280,<NA>,undefined,Item 2. Management&#8217;s Discussion and Ana...
1,NFLX,2021-09-30,https://www.sec.gov/Archives/edgar/data/106528...,10-Q,1065280,<NA>,undefined,Item 2. Management&#8217;s Discussion and Ana...
2,NFLX,2021-06-30,https://www.sec.gov/Archives/edgar/data/106528...,10-Q,1065280,<NA>,undefined,Item 2. Management&#8217;s Discussion and Ana...
3,NFLX,2021-03-31,https://www.sec.gov/Archives/edgar/data/106528...,10-Q,1065280,<NA>,undefined,Item 2. Management&#8217;s Discussion and Ana...
4,NFLX,2020-09-30,https://www.sec.gov/Archives/edgar/data/106528...,10-Q,1065280,<NA>,undefined,Item 2. \n\nManagement&#8217;s Discussion and...
...,...,...,...,...,...,...,...,...
511,MELI,2012-12-31,https://www.sec.gov/Archives/edgar/data/109959...,10-K,1099590,"ITEM 1. BUSINESS MercadoLibre, Inc. (together...",ITEM 1A. RISK FACTORS For purposes of this se...,ITEM 7. MANAGEMENT&#146;S DISCUSSION AND ANAL...
512,SOFI,2021-12-31,https://www.sec.gov/Archives/edgar/data/181887...,10-K/A,1818874,undefined,undefined,undefined
513,SOFI,2021-12-31,https://www.sec.gov/Archives/edgar/data/181887...,10-K,1818874,Item 1. Business \n\nCompany Overview \n\nWe ...,Item 1A. Risk Factors \n\nIn evaluating our c...,Item 7. MANAGEMENT&#8217;S DISCUSSION AND ANA...
514,SOFI,2020-12-31,https://www.sec.gov/Archives/edgar/data/181887...,10-K/A,1818874,Item 1. Business. \n\n&#160; \n\nOverview \n\...,Item 1.A. Risk Factors. \n\n&#160;\n\nAn inve...,Item 7. Management&#8217;s Discussion and Ana...


In [68]:
sec_text_df.to_csv('.\Data\Cleaned_Data\sec_all_data.csv')

In [69]:
def get_sec_text_df(ticker_list):
    file_10q_df = SEC_query(ticker_list,'10-Q')
    file_10k_df = SEC_query(ticker_list,'10-K')
    url_data = {
        "ticker":[],
        "periodOfReport":[],
        "linkToFilingDetails":[],
        "formType":[],
        "cik":[]
    }
    url_clean(file_10q_df)
    url_clean(file_10k_df)

    url_df = pd.DataFrame(url_data)

    return pd.concat([url_df,pull_sec_from_df(url_df)])